# Aprendizaje automático práctica 1

Lo primero de todo importar las librerías y los datos con los que se va a trabajar

In [ ]:
import numpy as np
import pandas as pd

data_path = 'train_test/attrition_availabledata_10.csv'
competition_path = 'train_test/attrition_competition_10.csv'

data = pd.read_csv(data_path, sep = ',')

data.head(10)

# EDA Simplificado:

Un EDA es una análisis exploratorio de datos, para organizar los datos, entender su contenido, entender cual son las variables más relevantes y cómo se relacionan unas con otras, determinar qué hacer con los datos faltantes y con los datos atípicos, y finalmente extraer conclusiones acerca de todo este análisis.

Para hacer un eda:

1.   Cuántas instancias y atributos hay.
2.   Qué tipo de atributos hay (numéricos o categóricos). Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding).
3.   Qué atributos tienen valores faltantes y cuántos.
4.   Si es un problema de clasificación o regresión (variable de respuesta) y, en caso de clasificación, si las clases están desbalanceadas.


In [ ]:
print('La forma de la tabla es:')
print('===============================')
print(data.shape)
print()

print('Los tipos de atributos son:')
print('================================')
data.info()

print()

print('Cuantos valores faltan por atributo:')
print('======================================')
print(data.isnull().sum())

print()

print('Fraccion de los valores que faltan por atributo:')
print('======================================')
print(data.isnull().mean())

print()

print('Comprobar si la clase está desbalanceada:')
print('======================================')
print(data['Attrition'].value_counts())
print()
print(data['Attrition'].value_counts() / data['Attrition'].count())

La forma de la tabla es:
(2940, 31)

Los tipos de atributos son:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      2940 non-null   float64
 1   absences                 2940 non-null   int64  
 2   JobInvolvement           2940 non-null   int64  
 3   PerformanceRating        2940 non-null   int64  
 4   EnvironmentSatisfaction  2925 non-null   float64
 5   JobSatisfaction          2928 non-null   float64
 6   WorkLifeBalance          2911 non-null   float64
 7   Age                      2940 non-null   int64  
 8   BusinessTravel           2940 non-null   object 
 9   Department               2940 non-null   object 
 10  DistanceFromHome         2940 non-null   int64  
 11  Education                2940 non-null   int64  
 12  EducationField           2940 non-null   object 
 13  EmployeeCount

Hay 8 variables no numéricas que hay que imputar 